# 2D Fourier Transform

In [3]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
images = [ cv2.imread("./data/00{}.jpg".format(i), cv2.IMREAD_GRAYSCALE) for i in range(1, 7)] 
cv2.imshow("i", images[1])
cv2.waitKey(0)

13

## Matrix Multiplication

In [135]:
class Matrix:
    def __init__(self, data):
        self.data = data
    
    def __repr__(self):
        return self.data.__repr__()
    
    def __getitem__(self, i):
        return self.data[i]
    
    def __truediv__(self, denom):
        div_mat = self.data.copy() 
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                div_mat[i][j] = div_mat[i][j] / denom 
        return Matrix(div_mat)
    
    @property
    def shape(self):
        return (len(self.data), len(self.data[0]))
    
    def round(self):
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                re, im = self.data[i][j].real, self.data[i][j].imag
                self.data[i][j] = round(re, 8) + 1j * round(im, 8)
    
    def matmul(self, B):
        result = [[0 for _ in range(B.shape[1])] for _ in range(self.shape[0])]
        for i in range(self.shape[0]):
            for j in range(B.shape[1]):
                sum = 0
                for k in range(self.shape[1]):
                    sum += self[i][k] * B[k][j]
                result[i][j] = sum 
        return Matrix(result)
    
    def transpose(self):
        result = [[0 for _ in range(self.shape[0])] for _ in range(self.shape[1])]
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                result[j][i] = self[i][j]
                
    @property
    def exp_mu(self):
        M = self.shape[0]
        result = [[0 for _ in range(M)] for _ in range(M)]
        for u in range(M):
            for m in range(M):
                result[m][u] = np.exp(-2j* np.pi * m * u / M )
        return Matrix(result)
    @property
    def exp_nu(self):
        N = self.shape[1]
        result = [[0 for _ in range(N)] for _ in range(N)]
        for v in range(N):
            for n in range(N):
                result[n][v] = np.exp(-2j* np.pi * n * v / N )
        return Matrix(result)
    
A = Matrix([[1, 2, 3], [4, 5, 6]])
B = Matrix([[1, 2], [3, 4], [5, 6]])

## 2D Fourier Transform

In [148]:
def fourier_2d(img: Matrix): 
    M, N = img.shape 
    fourier = img.exp_mu.matmul(img).matmul(img.exp_nu) 
    fourier.round()
    return fourier
    
    

In [149]:
print(A.exp_mu)
print(A.exp_nu)
k = A.exp_mu.matmul(A)
p = k.matmul(A.exp_nu)
print(p)

[[(1+0j), (1+0j)], [(1+0j), (-1-1.2246467991473532e-16j)]]
[[(1+0j), (1+0j), (1+0j)], [(1+0j), (-0.49999999999999983-0.8660254037844387j), (-0.5000000000000004+0.8660254037844384j)], [(1+0j), (-0.5000000000000004+0.8660254037844384j), (-0.4999999999999992-0.8660254037844392j)]]
[[(21+0j), (-3.000000000000002+1.732050807568874j), (-2.999999999999996-1.7320508075688839j)], [(-9-1.8369701987210296e-15j), (8.881784197001252e-16+1.3322676295501878e-15j), (-1.1102230246251565e-15+2.6645352591003757e-15j)]]


In [150]:
a = fourier_2d(Matrix(images[0]))

In [ ]:
a.data

[[(66.33608597+0j),
  (-18.62164428-3.68686112j),
  (-3.73563134-3.80009261j),
  (2.30823122+1.38336339j),
  (1.15981959+1.0441964j),
  (-0.98974825+0.38782493j),
  (-0.8242823+0.07193617j),
  (-0.17312591+0.2403765j),
  (0.32347878+0.26468153j),
  (0.42662382+0.25601436j),
  (0.22671136+0.39867924j),
  (-0.06926205+0.60907657j),
  (-0.09854844+0.25483206j),
  (-0.06537325+0.15968126j),
  (0.012281+0.1584346j),
  (0.04393188+0.19229639j),
  (0.00530953+0.22666029j),
  (-0.07240295+0.15775167j),
  (-0.04976754+0.16646151j),
  (-0.00272215+0.11580863j),
  (0.01530341+0.12309441j),
  (0.00554562+0.13228365j),
  (-0.02394771+0.17308572j),
  (-0.0378176+0.15720283j),
  (-0.02939302+0.11924801j),
  (-0.03123197+0.1085261j),
  (-0.02529766+0.07080481j),
  (0.00708926+0.08991377j),
  (0.00885394+0.09826612j),
  (-0.00593936+0.08330418j),
  (-0.00926879+0.1046115j),
  (-0.01156096+0.10089868j),
  (-0.00181475+0.10717075j),
  (-0.01904976+0.12987962j),
  (-0.03200848+0.11369553j),
  (-0.03381509

In [147]:
b = np.fft.fft2(images[0])
b

array([[ 6.45052100e+06     +0.j        ,
        -1.81076869e+06-358510.37551327j,
        -3.63252791e+05-369521.00585044j, ...,
         2.24452403e+05-134518.25566538j,
        -3.63252791e+05+369521.00585044j,
        -1.81076869e+06+358510.37551327j],
       [-1.12781460e+06-256690.12216659j,
         2.34476450e+05-460432.86490161j,
         4.74440241e+05+618894.5892061j , ...,
        -3.54098302e+05-176969.43118222j,
         2.70990317e+05-408490.6023802j ,
         2.37613074e+05-174682.77095576j],
       [-1.17115301e+05-499075.19373302j,
         1.82997532e+05+106634.37018425j,
        -3.91914362e+03 +31039.63578145j, ...,
         1.61456109e+05+357716.98596997j,
        -1.84818004e+05-306781.99533241j,
         4.13450826e+05-298398.54516473j],
       ...,
       [-6.09638567e+05+185655.85730448j,
         2.84201452e+05+364306.50572386j,
        -1.76640744e+05-368998.42652829j, ...,
         1.19473283e+05 +58728.02714679j,
         8.87406310e+04 +16969.259399j  ,

In [145]:
for i in range(10):
    print(a.data[0][i].real, b[0][i].real)
    print(a.data[0][i].imag, b[0][i].imag)

66.33608597 6450521.0
0.0 0.0
-18.62164428 -1810768.689584615
-3.68686112 -358510.375513272
-3.73563134 -363252.7914773805
-3.80009261 -369521.0058504423
2.30823122 224452.4033507025
1.38336339 134518.25566537798
1.15981959 112780.8570862262
1.0441964 101537.65760747004
-0.98974825 -96243.11962743022
0.38782493 37712.09653793356
-0.8242823 -80153.21044439549
0.07193617 6995.0729050270065
-0.17312591 -16834.76389424751
0.2403765 23374.210588113354
0.32347878 31455.077051596785
0.26468153 25737.632340806726
0.42662382 41484.90036671821
0.25601436 24894.83654820084


In [133]:
np.fft.fft2(B.data)

array([[21.+0.j        , -3.+0.j        ],
       [-6.+3.46410162j,  0.+0.j        ],
       [-6.-3.46410162j,  0.+0.j        ]])